In [1]:
"""
ucd_atlas_loop.py
create and send sql query looping over atlas3d galaxies to find CEs
based on code from Aaron Deich
"""

from numpy import *
import urllib, urllib2

# Formulas:
# modulus = 5 * log10(a_dist * 1e6) - 5
# M_r = r - modulus
# size = petroR50_r / 3600 / 180 * pi * a_dist * 1e3

def writer(name,host,data):
    # Function to write data to a file
    f = open(name, 'a')             #defines the file to be opened as appendable
    #f.write('\n' + host + '\n')     #formatting for hosts
    f.write(data)                   #writes in data
    f.close()                       #tells python to close file
    return writer

def create_SQL_query(RA, DEC, distance, radius):
    # creates SQL query to search a radius (in kpc) around an object
    query_str = 'SELECT p.objID,p.ra,p.dec '#,p.u,p.g,p.r,p.i,p.z,p.petroR90_r / p.petroR50_r as Concentration, p.expAB_r, p.deVAB_r '
    query_str = ''.join([query_str, 'FROM PhotoObj p JOIN dbo.fGetNearbyObjEq('])
    query_str = ''.join([query_str, str(RA), ',', str(DEC), ',', str(radius), '*60*180/3.15159/1000/', str(distance)])
    query_str = ''.join([query_str, ') AS GN ON p.objID = GN.objID WHERE p.g-p.i between 0.9 and 1.4 AND p.type = 3 AND '])
    query_str = ''.join([query_str, '(p.petroR50_r'])
    #query_str = ''.join([query_str, '(((p.deVRad_r * p.fracDeV_r + p.expRad_r * (1 - p.fracDeV_r)))'])
    query_str = ''.join([query_str, '/3600/180*3.14159*1000*'])
    query_str = ''.join([query_str, str(distance), ') between 0.04 and 1'])
    query_str = ''.join([query_str, ' AND p.r - (5*log10(', str(distance), '*1000000)) + 5 between -20 and -12'])
    query_str = ''.join([query_str, ' AND p.petroR90_r / p.petroR50_r between 2.15 AND 2.95'])
    #query_str = ''.join([query_str, ' AND p.expAB_r between 0.77 AND 1.0'])
    query_str = ''.join([query_str, ' AND p.deVAB_r between 0.81 AND 1.0'])
    query_str = ''.join([query_str, " AND (flags & dbo.fPhotoFlags('SATUR_CENTER')) = 0"])
    return query_str

def SDSS_request(SQL_query):
    # sends SQL query to DR7 SQL search, returns as csv
    sURL = 'http://cas.sdss.org/dr7/en/tools/search/x_sql.asp'
    values = {'cmd': SQL_query, 'format': 'csv'}
    data = urllib.urlencode(values)
    request = urllib2.Request(sURL, data)
    response = urllib2.urlopen(request)
    return response.read()

# gets locations of and distances to objects in Atlas3D database
host_obj=genfromtxt('cappellari_early_type.csv',delimiter=',',skip_header=1,dtype=None, names=['name','ra','dec','sbf','ned_d','virgo','v_hel','distance','M_K','A_B','T_type','log(re)'])

# search radius of 40kpc, with search_rad in arcmin
# radius = 40 * 1e3 * 60 * 180 / pi / distance
radius = 40 #kpc
print ''
n = 5 # IC1024

#--------------------------------------------------------------------
# Section for testing single host galaxies
#--------------------------------------------------------------------
# create sql query
#query_string = create_SQL_query(host_obj['ra'][n], host_obj['dec'][n], host_obj['distance'][n], radius)
# send SQL query to DR7 database
#data_string = SDSS_request(query_string)
#print name[n]
#print query_string, '\n\n', data_string

file_name = raw_input('Enter a name for your .csv file: ') + '.csv'
#file_name = 'ic1024_test'
op = open(file_name, 'w')
op.write('Complete list of found objects sorted by host galaxy; by Alex Colebaugh \n')
#op.write(data_string)
op.close()

query_string = ''
data_string = ''
full_data = ''

for obj in arange(0,len(host_obj['ra']),1):
#for obj in arange(0,10,1):
    query_string = create_SQL_query(host_obj['ra'][obj], host_obj['dec'][obj], host_obj['distance'][obj], radius) 
    #print sql_query_string
    data_string = SDSS_request(query_string)
    print host_obj['name'][obj]
    print data_string
    if data_string == 'No objects have been found':
        continue 
    else:
        #split_string = data_string.split('deVAB_r\n')[1]
        split_string = data_string.split('dec\n')[1]
        if split_string in full_data:
            continue
        else:
            full_data += split_string
writer(file_name,host_obj['name'][obj],full_data)

# ucds_concentration: fracDeV, concentration between 2.0 and 3.0, cuts 914 entries but also UCDs
# ucds_petro_conc: petro, concentration between 2.0 and 3.0 cuts 795 entries
# ucds_restricted: petro, concentration between 2.15 and 2.95 cuts 1249 entries
# ucds_expfit: petro, concentration between 2.15 and 2.95 cuts 2272 entries


Enter a name for your .csv file: test
IC0560
No objects have been found
IC0598
No objects have been found
IC0676
objID,ra,dec
587734892211142882,168.25459112,9.07823679

IC0719
No objects have been found
IC0782
objID,ra,dec
587742955471176176,185.42792931,5.79226996
587742955471175835,185.44780669,5.72155169

IC1024
No objects have been found
IC3631
objID,ra,dec
588017571384721868,189.93878694,13.01619281

NGC0448
No objects have been found
NGC0474
No objects have been found
NGC0502
No objects have been found
NGC0509
No objects have been found
NGC0516
No objects have been found
NGC0524
No objects have been found
NGC0525
No objects have been found
NGC0661
No objects have been found
NGC0680
No objects have been found
NGC0770
No objects have been found
NGC0821
No objects have been found
NGC0936
objID,ra,dec
588015507674890334,36.9407491,-1.15392902
588015507674955924,36.96891554,-1.15764169
588015507674890372,36.83676865,-1.12947894
588015507674955948,36.98089839,-1.09167401

NGC1023
No 

NGC4434
objID,ra,dec
588017726011932802,186.97846063,8.09038648
588017730299756778,186.80163593,8.14424409

NGC4435
No objects have been found
NGC4442
objID,ra,dec
587734892756074658,186.90304867,9.83382551
587732771057500279,187.09751815,9.71340349

NGC4452
objID,ra,dec
588017565490741351,187.14462329,11.69077619
588017565490675889,187.09207807,11.76856493
588017565490675941,187.09024554,11.81907137
588017565490741373,187.17223832,11.85390812
588017565490807006,187.29241456,11.7542713
588017565490675765,187.06201266,11.78982472

NGC4458
objID,ra,dec
588017704544174136,187.23983995,13.24192595
588017704544174201,187.18244118,13.28638091

NGC4459
No objects have been found
NGC4461
objID,ra,dec
588017704544174136,187.23983995,13.24192595
588017704544174201,187.18244118,13.28638091

NGC4472
objID,ra,dec
588017726012129449,187.32897396,8.01965086
588017726012194878,187.5027901,8.10433257
588017726012129348,187.37603938,7.99286113
588017726012129381,187.39581088,8.05800197
58801772601212950

PGC016060
No objects have been found
PGC028887
objID,ra,dec
587735347486458081,149.90902016,11.65800044
587735347486458111,149.95632281,11.65715641
587735347486458369,149.88839027,11.63487357

PGC029321
objID,ra,dec
587738410862837838,151.46329329,12.96129752

PGC035754
No objects have been found
PGC042549
objID,ra,dec
587745545880142141,190.32234279,-5.05050812

PGC044433
No objects have been found
PGC050395
No objects have been found
PGC051753
No objects have been found
PGC054452
No objects have been found
PGC056772
No objects have been found
PGC058114
No objects have been found
PGC061468
No objects have been found
PGC071531
No objects have been found
PGC170172
No objects have been found
UGC03960
objID,ra,dec
587732053233893641,115.13274155,23.32825568
587732053233894287,115.16479679,23.29583834

UGC04551
objID,ra,dec
588010137338183974,131.07701321,49.78813123

UGC05408
No objects have been found
UGC06062
No objects have been found
UGC06176
objID,ra,dec
587742061602472502,166.826414

<function __main__.writer>

In [ ]:

objID,ra,dec
